In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-18"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
17858,2024-12-18,República Tcheca Copa Tcheca Feminina,13:00,BK Havirov F,Hradec Kralove F,1.67,2.02,143.5,1.83,1.83,20.5,1.98,1.70,2V3BUdfM,0.598802,0.495050,0.546448,0.546448,0.093852,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,79,79,1.04,1.22,0.000,0.000,0.134139,0.000000,0.107603,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17859,2024-12-18,Japão B2.League,07:00,Shizuoka,Kobe,1.52,2.30,158.5,1.82,1.84,-5.5,2.01,1.68,pzJbhaUH,0.657895,0.434783,0.549451,0.543478,0.092677,175.208,112.459718,0.641864,1.8,1.643168,0.912871,106.65,2.304,1.325549,0.575325,2.0,361.500,152.800753,0.422685,0.0,0.000000,NaN,205.92,4.710,1.976752,0.419693,-89.0,79,72,1.35,2.86,214.232,220.550,0.288766,0.007728,0.126474,-0.90,-0.180,-2.888889,0.499089,0.4,-0.099089,-5.00,-1.000,-1.300000,0.540629,0.1,-0.440629
17860,2024-12-18,Japão B2.League,07:05,Fukuoka,Shinshu,1.52,2.29,159.5,1.82,1.84,-5.5,2.03,1.67,bwqY6bqB,0.657895,0.436681,0.549451,0.543478,0.094576,113.058,12.567327,0.111158,3.0,0.000000,0.000000,120.96,1.316,0.243783,0.185245,75.0,160.502,72.232097,0.450039,1.8,1.643168,0.912871,242.55,1.898,1.184449,0.624051,73.0,84,77,1.44,3.15,118.096,173.884,0.285812,0.007728,0.137599,1.38,0.276,1.884058,0.693278,0.8,0.106722,-0.51,-0.102,-12.647059,0.518889,0.6,0.081111
17861,2024-12-18,Japão B2.League,07:05,Fukushima,Toyama,7.70,1.04,158.5,1.81,1.85,9.5,1.96,1.75,2BkP8KEb,0.129870,0.961538,0.552486,0.540541,0.091409,437.262,366.827898,0.838920,0.6,1.341641,2.236068,1008.00,6.570,5.327185,0.810835,-54.0,124.512,45.655424,0.366675,1.8,1.643168,0.912871,204.12,1.554,0.676447,0.435294,29.0,70,81,14.40,2.52,204.572,113.026,1.077650,0.015456,0.080050,-2.59,-0.518,-12.934363,0.403040,0.3,-0.103040,1.37,0.274,0.145985,0.688042,0.7,0.011958
17862,2024-12-18,Polônia 1. Liga,13:00,Trefl Sopot II,Kotwica Kolobrzeg,2.95,1.33,162.5,1.88,1.78,4.5,1.95,1.72,SfAsu7Pr,0.338983,0.751880,0.531915,0.561798,0.090863,197.724,62.210704,0.314634,0.6,1.341641,2.236068,128.16,2.384,0.620669,0.260348,-28.0,122.068,39.140531,0.320645,2.4,1.341641,0.559017,177.16,1.612,0.401086,0.248813,25.0,72,86,1.78,2.06,137.952,159.240,0.535286,0.038640,0.088629,-2.98,-0.596,-3.271812,0.000000,0.0,0.000000,-2.16,-0.432,-0.763889,0.528061,0.5,-0.028061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18044,2024-12-18,Europa Liga Da Aba 2,14:00,Rabotnicki,Vojvodina Novi Sad,9.35,1.01,157.5,1.86,1.80,16.5,1.93,1.74,M1mKOKZ9,0.106952,0.990099,0.537634,0.555556,0.097051,432.800,229.137996,0.529432,0.0,0.000000,NaN,660.00,6.628,3.510060,0.529581,-114.0,119.412,34.181328,0.286247,2.4,1.341641,0.559017,144.20,1.424,0.413437,0.290335,54.0,75,70,8.80,2.06,0.000,0.000,1.138469,0.023184,0.073215,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
18045,2024-12-18,Europa Liga Da Aba 2,16:00,Sutjeska,KK Bosna,1.54,2.25,156.5,1.84,1.82,-4.5,1.93,1.74,QZgBQt5c,0.649351,0.444444,0.543478,0.549451,0.093795,169.996,85.648531,0.503827,1.8,1.643168

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
17906,21:00,Eua Ncaa,Vermont,Miami (Ohio),1.55,Back Home
17911,21:30,Eua Ncaa,UTRGV,Southern Utah,1.69,Back Home
17924,11:00,Rússia Superliga,University-Ugra,Dynamo Vladivostok,2.68,Back Home
17950,12:00,Rússia Superliga,Dome-Springs Izhevsk,Chelyabinsk,1.83,Back Home
17973,20:30,Eua Ncaa,Georgetown,Creighton,2.47,Back Home
17986,15:00,Europa Eurocopa,Ulm,BC Wolves,1.57,Back Home
18001,21:00,Eua Ncaa,Virginia,Memphis,2.61,Back Home
18028,22:00,Eua Ncaa,La Lafayette,Appalachian State,2.18,Back Home
18041,17:00,Europa Liga Dos Campeões,Saint Quentin,Karsiyaka,1.97,Back Home
